In [25]:
#Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from multiprocessing import Pool


import torchvision
from torchvision import models as tvmodels
from torchsummary import summary

import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, Dataset

import torchvision.models as torchvisionmodels

import os
import numpy as np
import cv2
import argparse
import sys

import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
import itertools
import more_itertools

import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt

from captum.attr import LayerGradCam
from captum.attr import visualization
from PIL import Image
import shutil

import numpy as np
from dask_image.imread import imread
from dask_image import ndfilters, ndmorph, ndmeasure
import matplotlib.pyplot as plt
from dask_image import ndmeasure

from operator import itemgetter


from torchattacks import *
import torchattacks

We will test on two attacks that were trained on
1. BIM
2. Square


We will test on two attacks that were not trained on
1. Autoattack / DeepFool
2. CW

In [2]:
#Autoattack robustness testing



## NSFW Evaluations

In [2]:
from model import resnet
#from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
# model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
# checkpoint = torch.load('nsfw-pytorch/models/nsfw/epoch_2.pkl')
# load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2, bias=True)
checkpoint = torch.load('non_robust_models/ResNet_NSFW.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'neutral', 1:'porn'}

test_path = 'nsfw/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /home/ocistudent/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [81]:
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    #adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.9801324503311258


In [3]:
torch.manual_seed(0)

#labels = 2

#atk = torchattacks.PGD(model, eps=16/255, alpha=2/255, steps=100, random_start=True)
#atk = torchattacks.Square(model, eps=16/255, n_queries=500, n_restarts=1, loss='ce')
#atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
#atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
#atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=100, random_start=True)
#atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
#atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
#atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
#adv_image = atk(images, labels)

In [4]:
# Attack 1
atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.25165562913907286


In [5]:
# Attack 2
atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.6357615894039735


In [6]:
# Attack 3
atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.12582781456953643


In [7]:
# Attack 4
atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.6490066225165563


In [9]:
# Attack 5
atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):

    inputs, targets = inputs, targets
    #print(inputs.size())
    try:
        adv_image = atk(inputs, targets)
    
    except:
        pass
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.0


In [9]:
# Combination

correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
    adv_image1 = atk1(inputs, targets)
    atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
    adv_image2 = atk2(inputs, targets)
    
    add = adv_image1.add(adv_image2)
    
    adv_image = torch.div(add, 2)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.5960264900662252


## CB Evaluations

In [17]:
from model import resnet
#from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
# model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
# checkpoint = torch.load('nsfw-pytorch/models/nsfw/epoch_2.pkl')
# load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2, bias=True)
checkpoint = torch.load('non_robust_models/ResNet_CB.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'cyberbullying', 1:'non_cyberbullying'}

test_path = 'cyberbullying/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /home/ocistudent/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [83]:
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    #adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.9139072847682119


In [9]:
torch.manual_seed(0)

#labels = 2

#atk = torchattacks.PGD(model, eps=16/255, alpha=2/255, steps=100, random_start=True)
#atk = torchattacks.Square(model, eps=16/255, n_queries=500, n_restarts=1, loss='ce')
#atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
#atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=100, random_start=True)
#adv_image = atk(images, labels)

In [10]:
# Attack 1
atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.48344370860927155


In [11]:
# Attack 2
atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

/home/ocistudent/ocistudent/envs/SMU/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


0.5695364238410596


In [12]:
# Attack 3
atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.052980132450331126


In [13]:
# Attack 4
atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.7682119205298014


In [13]:
# Attack 5
atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):

    inputs, targets = inputs, targets
    #print(inputs.size())
    try:
        adv_image = atk(inputs, targets)
    
    except:
        pass
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.019867549668874173


In [11]:
# Combination

correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
    adv_image1 = atk1(inputs, targets)
    atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
    adv_image2 = atk2(inputs, targets)
    
    add = adv_image1.add(adv_image2)
    
    adv_image = torch.div(add, 2)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.695364238410596


## Self-Harm Evaluations

In [18]:
from model import resnet
#from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
# model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
# checkpoint = torch.load('nsfw-pytorch/models/nsfw/epoch_2.pkl')
# load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2, bias=True)
checkpoint = torch.load('non_robust_models/ResNet_SH.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'neutral', 1:'porn'}

test_path = 'self_harm/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /home/ocistudent/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [85]:
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    #adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.9470198675496688


In [15]:
torch.manual_seed(0)

#labels = 2

#atk = torchattacks.PGD(model, eps=16/255, alpha=2/255, steps=100, random_start=True)
#atk = torchattacks.Square(model, eps=16/255, n_queries=500, n_restarts=1, loss='ce')
#atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
#atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=100, random_start=True)
#adv_image = atk(images, labels)

In [16]:
# Attack 1
atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.5231788079470199


In [17]:
# Attack 2
atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.5364238410596026


In [18]:
# Attack 3
atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.052980132450331126


In [19]:
# Attack 4
atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    adv_image = atk(inputs, targets)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.5231788079470199


In [15]:
# Attack 5
atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):

    inputs, targets = inputs, targets
    #print(inputs.size())
    try:
        adv_image = atk(inputs, targets)
    
    except:
        pass
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.052980132450331126


In [13]:
# Combination

correct = 0
total_samples = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(type(targets.item()))
    atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
    adv_image1 = atk1(inputs, targets)
    atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
    adv_image2 = atk2(inputs, targets)
    
    add = adv_image1.add(adv_image2)
    
    adv_image = torch.div(add, 2)
    
    label = targets.item()
        
    outputs = model(adv_image.to(device))
    _, predicted = outputs.max(1)
    pred_val = predicted.item()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print(correct / total_samples)

0.7218543046357616


## Create Datasets of Attacked Images to be used in Purification Tests

### NSFW

In [19]:
from model import resnet
#from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
# model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
# checkpoint = torch.load('nsfw-pytorch/models/nsfw/epoch_2.pkl')
# load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2, bias=True)
checkpoint = torch.load('non_robust_models/ResNet_NSFW.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'neutral', 1:'porn'}

test_path = 'nsfw/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /home/ocistudent/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [72]:
i = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))
        if targets.item() == 0:
            torchvision.utils.save_image(inputs, 'non_robust_models/NSFW_attacked/clean'+'/neutral/'+str(i)+'.png')
            
        if targets.item() == 1:
            torchvision.utils.save_image(inputs, 'non_robust_models/NSFW_attacked/clean'+'/porn/'+str(i)+'.png')
         
        i += 1
        if i > 400:
            break

In [23]:
i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))
        adv_image = atk(inputs, targets)
        if targets.item() == 0:
            #BIM
            atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/BIM'+'/neutral/'+str(i)+'.png')
            #Square
            atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/Square'+'/neutral/'+str(i)+'.png')
            #DF
            atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/DF'+'/neutral/'+str(i)+'.png')
            #CW
            atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/CW'+'/neutral/'+str(i)+'.png')
            
        if targets.item() == 1:
            #BIM
            atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/BIM'+'/porn/'+str(i)+'.png')
            #Square
            atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/Square'+'/porn/'+str(i)+'.png')
            #DF
            atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/DF'+'/porn/'+str(i)+'.png')
            #CW
            atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/CW'+'/porn/'+str(i)+'.png')
                    
        i += 1
        if i > 400:
            break

In [11]:
#Auto

i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))

        #adv_image = atk(inputs, targets)
        if targets.item() == 0:
            #Auto
            atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
            try:
                adv_image = atk(inputs, targets)
            except:
                pass
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/auto'+'/neutral/'+str(i)+'.png')

        
        
        if targets.item() == 1:
            #Auto
            atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
            try:
                adv_image = atk(inputs, targets)
            except:
                pass
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/auto'+'/porn/'+str(i)+'.png')

                    
        i += 1
        if i > 400:
            break

In [20]:
# Combination

i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))

        #adv_image = atk(inputs, targets)
        if targets.item() == 0:
            atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image1 = atk1(inputs, targets)
            atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image2 = atk2(inputs, targets)
    
            add = adv_image1.add(adv_image2)
    
            adv_image = torch.div(add, 2)
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/comb'+'/neutral/'+str(i)+'.png')

        
        
        if targets.item() == 1:
            atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image1 = atk1(inputs, targets)
            atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image2 = atk2(inputs, targets)
    
            add = adv_image1.add(adv_image2)
    
            adv_image = torch.div(add, 2)
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/NSFW_attacked/comb'+'/porn/'+str(i)+'.png')

                    
        i += 1
        if i > 400:
            break

### Cyberbullying

In [21]:
from model import resnet
#from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
# model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
# checkpoint = torch.load('nsfw-pytorch/models/nsfw/epoch_2.pkl')
# load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2, bias=True)
checkpoint = torch.load('non_robust_models/ResNet_CB.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'cyberbullying', 1:'non_cyberbullying'}

test_path = 'cyberbullying/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /home/ocistudent/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [74]:
i = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))
        if targets.item() == 0:
             torchvision.utils.save_image(inputs, 'non_robust_models/cyberbullying_attacked/clean'+'/cyberbullying/'+str(i)+'.png')
            
        if targets.item() == 1:
             torchvision.utils.save_image(inputs, 'non_robust_models/cyberbullying_attacked/clean'+'/non_cyberbullying/'+str(i)+'.png')
         
        i += 1
        if i > 400:
            break

In [25]:
i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))
        adv_image = atk(inputs, targets)
        if targets.item() == 1:
            #BIM
            atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/BIM'+'/non_cyberbullying/'+str(i)+'.png')
            #Square
            atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/Square'+'/non_cyberbullying/'+str(i)+'.png')
            #DF
            atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/DF'+'/non_cyberbullying/'+str(i)+'.png')
            #CW
            atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/CW'+'/non_cyberbullying/'+str(i)+'.png')
            
        if targets.item() == 0:
            #BIM
            atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/BIM'+'/cyberbullying/'+str(i)+'.png')
            #Square
            atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/Square'+'/cyberbullying/'+str(i)+'.png')
            #DF
            atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/DF'+'/cyberbullying/'+str(i)+'.png')
            #CW
            atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/CW'+'/cyberbullying/'+str(i)+'.png')
                    
        i += 1
        if i > 400:
            break

/home/ocistudent/ocistudent/envs/SMU/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [17]:
# auto

i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))

        #adv_image = atk(inputs, targets)
        if targets.item() == 0:
            #Auto
            atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
            try:
                adv_image = atk(inputs, targets)
            except:
                pass
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/auto'+'/cyberbullying/'+str(i)+'.png')

        
        
        if targets.item() == 1:
            #Auto
            atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
            try:
                adv_image = atk(inputs, targets)
            except:
                pass
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/auto'+'/non_cyberbullying/'+str(i)+'.png')

                    
        i += 1
        if i > 400:
            break

/home/ocistudent/ocistudent/envs/SMU/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [22]:
# comb

i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))

        #adv_image = atk(inputs, targets)
        if targets.item() == 0:
            atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image1 = atk1(inputs, targets)
            atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image2 = atk2(inputs, targets)
    
            add = adv_image1.add(adv_image2)
    
            adv_image = torch.div(add, 2)
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/comb'+'/cyberbullying/'+str(i)+'.png')

        
        
        if targets.item() == 1:
            atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image1 = atk1(inputs, targets)
            atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image2 = atk2(inputs, targets)
    
            add = adv_image1.add(adv_image2)
    
            adv_image = torch.div(add, 2)
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/cyberbullying_attacked/comb'+'/non_cyberbullying/'+str(i)+'.png')

                    
        i += 1
        if i > 400:
            break

/home/ocistudent/ocistudent/envs/SMU/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


### Self-Harm

In [23]:
from model import resnet
#from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
# model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
# checkpoint = torch.load('nsfw-pytorch/models/nsfw/epoch_2.pkl')
# load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2, bias=True)
checkpoint = torch.load('non_robust_models/ResNet_SH.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'non_self_harm', 1:'self_harm'}

test_path = 'self_harm/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /home/ocistudent/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [76]:
i = 0

for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))
        if targets.item() == 0:
             torchvision.utils.save_image(inputs, 'non_robust_models/self_harm_attacked/clean'+'/non_self_harm/'+str(i)+'.png')
            
        if targets.item() == 1:
             torchvision.utils.save_image(inputs, 'non_robust_models/self_harm_attacked/clean'+'/self_harm/'+str(i)+'.png')
         
        i += 1
        if i > 400:
            break

In [27]:
i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))
        adv_image = atk(inputs, targets)
        if targets.item() == 0:
            #BIM
            atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/BIM'+'/non_self_harm/'+str(i)+'.png')
            #Square
            atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/Square'+'/non_self_harm/'+str(i)+'.png')
            #DF
            atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/DF'+'/non_self_harm/'+str(i)+'.png')
            #CW
            atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/CW'+'/non_self_harm/'+str(i)+'.png')
            
        if targets.item() == 1:
            #BIM
            atk = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/BIM'+'/self_harm/'+str(i)+'.png')
            #Square
            atk = torchattacks.Square(model, norm='Linf', n_queries=500, n_restarts=1, eps=16/255, p_init=.8, seed=0, verbose=False, loss='margin', resc_schedule=True)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/Square'+'/self_harm/'+str(i)+'.png')
            #DF
            atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/DF'+'/self_harm/'+str(i)+'.png')
            #CW
            atk = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
            adv_image = atk(inputs, targets)
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/CW'+'/self_harm/'+str(i)+'.png')
                    
        i += 1
        if i > 400:
            break

In [19]:
# auto

i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))

        #adv_image = atk(inputs, targets)
        if targets.item() == 0:
            #Auto
            atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
            try:
                adv_image = atk(inputs, targets)
            except:
                pass
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/auto'+'/non_self_harm/'+str(i)+'.png')

        
        
        if targets.item() == 1:
            #Auto
            atk = torchattacks.AutoAttack(model, eps=8/255, n_classes=2, version='standard')
            try:
                adv_image = atk(inputs, targets)
            except:
                pass
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/auto'+'/self_harm/'+str(i)+'.png')

                    
        i += 1
        if i > 400:
            break

In [24]:
# comb

i = 0


for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        #print(type(targets.item()))

        #adv_image = atk(inputs, targets)
        if targets.item() == 0:
            atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image1 = atk1(inputs, targets)
            atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image2 = atk2(inputs, targets)
    
            add = adv_image1.add(adv_image2)
    
            adv_image = torch.div(add, 2)
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/comb'+'/non_self_harm/'+str(i)+'.png')

        
        
        if targets.item() == 1:
            atk1 = torchattacks.BIM(model, eps=4/255, alpha=1/255, steps=10)
            adv_image1 = atk1(inputs, targets)
            atk2 = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
            adv_image2 = atk2(inputs, targets)
    
            add = adv_image1.add(adv_image2)
    
            adv_image = torch.div(add, 2)
            
            torchvision.utils.save_image(adv_image, 'non_robust_models/self_harm_attacked/comb'+'/self_harm/'+str(i)+'.png')

                    
        i += 1
        if i > 400:
            break

## Robust Model Testing

### NSFW

In [91]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2, bias=True)
checkpoint = torch.load('robust_models/robust_NSFW.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'neutral', 1:'porn'}

test_path = 'nsfw/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /home/ocistudent/.cache/torch/hub/pytorch_vision_v0.10.0


_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=['fc.fc.weight', 'fc.fc.bias'])

In [26]:
from modules import *
model = resnet50()
checkpoint = torch.load('robust_models/robust_NSFW.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor()])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'neutral', 1:'porn'}

test_path = 'nsfw/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

<All keys matched successfully>

In [93]:
total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on unattacked data: '+str(correct / total_samples))

Accuracy of robust model on unattacked data: 0.9006622516556292


In [65]:
test_path = 'non_robust_models/NSFW_attacked/Square'

transform_test = transforms.Compose([transforms.ToTensor()])

testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on BIM: '+str(correct / total_samples))

Accuracy of robust model on BIM: 0.9403973509933775


### Cyberbullying

In [100]:
from model import resnet
from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
checkpoint = torch.load('robust_models/robust_CB.pth')
load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'non_self_harm', 1:'self_harm'}

test_path = 'cyberbullying/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
#model.load_state_dict(checkpoint, strict=False)

320 320


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [102]:
total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(targets)
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on unattacked data: '+str(correct / total_samples))

/home/ocistudent/ocistudent/envs/SMU/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Accuracy of robust model on unattacked data: 0.8874172185430463


In [52]:
test_path = 'non_robust_models/self_harm_attacked/BIM'

transform_test = transforms.Compose([transforms.ToTensor()])

testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on BIM: '+str(correct / total_samples))

Accuracy of robust model on BIM: 0.9205298013245033


### Self-Harm

In [103]:
from model import resnet
from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
checkpoint = torch.load('robust_models/robust_SH.pth')
load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'non_self_harm', 1:'self_harm'}

test_path = 'self_harm/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
#model.load_state_dict(checkpoint, strict=False)

320 320


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [105]:
total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    #print(targets)
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on unattacked data: '+str(correct / total_samples))

Accuracy of robust model on unattacked data: 0.9139072847682119


In [52]:
test_path = 'non_robust_models/self_harm_attacked/BIM'

transform_test = transforms.Compose([transforms.ToTensor()])

testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on BIM: '+str(correct / total_samples))

Accuracy of robust model on BIM: 0.9205298013245033


## Evaluation of Robust Models with Purified Images

### NSFW

In [27]:
from modules import *
model = resnet50()
checkpoint = torch.load('robust_models/robust_NSFW.pth')
#model.load_state_dict(checkpoint, strict=False)
transform_test = transforms.Compose([transforms.ToTensor()])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'neutral', 1:'porn'}

test_path = 'nsfw/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

<All keys matched successfully>

In [37]:
test_path = 'purified_images/NSFW_purified/clean'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified clean: '+str(correct / total_samples))

Accuracy of robust model on Purified clean: 0.8741721854304636


In [11]:
test_path = 'purified_images/NSFW_purified/BIM'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified BIM: '+str(correct / total_samples))

Accuracy of robust model on Purified BIM: 0.8874172185430463


In [12]:
test_path = 'purified_images/NSFW_purified/Square'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified Square: '+str(correct / total_samples))

Accuracy of robust model on Purified Square: 0.8807947019867549


In [47]:
test_path = 'purified_images/NSFW_purified/CW'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified CW: '+str(correct / total_samples))

Accuracy of robust model on Purified CW: 0.7417218543046358


In [45]:
test_path = 'purified_images/NSFW_purified/DF'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified DF: '+str(correct / total_samples))

Accuracy of robust model on Purified DF: 0.8807947019867549


In [22]:
test_path = 'purified_images/NSFW_purified/auto'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified auto: '+str(correct / total_samples))

Accuracy of robust model on Purified auto: 0.8410596026490066


In [34]:
test_path = 'purified_images/NSFW_purified/comb'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified comb: '+str(correct / total_samples))

Accuracy of robust model on Purified comb: 0.8940397350993378


### Cyberbullying

In [35]:
from model import resnet
from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
checkpoint = torch.load('robust_models/robust_CB.pth')
load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
transform_test = transforms.Compose([transforms.ToTensor()])

good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'non_self_harm', 1:'self_harm'}

test_path = 'cyberbullying/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
#model.load_state_dict(checkpoint, strict=False)

320 320


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
test_path = 'purified_images/cyberbullying_purified/clean'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified clean: '+str(correct / total_samples))

Accuracy of robust model on Purified clean: 0.9602649006622517


In [3]:
test_path = 'purified_images/cyberbullying_purified/BIM'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified BIM: '+str(correct / total_samples))

Accuracy of robust model on Purified BIM: 0.9470198675496688


In [7]:
test_path = 'purified_images/cyberbullying_purified/Square'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified Square: '+str(correct / total_samples))

Accuracy of robust model on Purified Square: 0.9536423841059603


In [8]:
test_path = 'purified_images/cyberbullying_purified/CW'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified CW: '+str(correct / total_samples))

Accuracy of robust model on Purified CW: 0.8344370860927153


In [9]:
test_path = 'purified_images/cyberbullying_purified/DF'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified DF: '+str(correct / total_samples))

Accuracy of robust model on Purified DF: 0.9271523178807947


In [5]:
test_path = 'purified_images/cyberbullying_purified/auto'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified auto: '+str(correct / total_samples))

Accuracy of robust model on Purified auto: 0.9072847682119205


In [38]:
test_path = 'purified_images/cyberbullying_purified/comb'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified comb: '+str(correct / total_samples))

Accuracy of robust model on Purified comb: 0.9337748344370861


### Self-harm

In [39]:
from model import resnet
from model.utils import load_filtered_state_dict, SaveBestModel, AverageMeter, accuracy
model = resnet.ResNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 2)
checkpoint = torch.load('robust_models/robust_SH.pth')
load_filtered_state_dict(model, checkpoint, ignore_layer=[], reverse=True)
transform_test = transforms.Compose([transforms.ToTensor()])


device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'non_self_harm', 1:'self_harm'}


model.to(device)
model.eval()
#model.load_state_dict(checkpoint, strict=False)

320 320


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
test_path = 'purified_images/self_harm_purified/clean'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified clean: '+str(correct / total_samples))

Accuracy of robust model on Purified clean: 0.9072847682119205


In [12]:
test_path = 'purified_images/self_harm_purified/BIM'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified BIM: '+str(correct / total_samples))

Accuracy of robust model on Purified BIM: 0.8874172185430463


In [13]:
test_path = 'purified_images/self_harm_purified/Square'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified Square: '+str(correct / total_samples))

Accuracy of robust model on Purified Square: 0.9006622516556292


In [14]:
test_path = 'purified_images/self_harm_purified/CW'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified CW: '+str(correct / total_samples))

Accuracy of robust model on Purified CW: 0.7019867549668874


In [15]:
test_path = 'purified_images/self_harm_purified/DF'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified DF: '+str(correct / total_samples))

Accuracy of robust model on Purified DF: 0.9072847682119205


In [7]:
test_path = 'purified_images/self_harm_purified/auto'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified auto: '+str(correct / total_samples))

Accuracy of robust model on Purified auto: 0.8675496688741722


In [41]:
test_path = 'purified_images/self_harm_purified/comb'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

total_samples = 0
correct = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs, targets
    
    label = targets
        
    outputs = model(inputs.to(device))
    _, predicted = outputs.max(1)
    
    
    pred_val = predicted.cpu()
    
    total_samples += 1
    if pred_val == label:
        correct += 1
    
    if total_samples > 150:
        break
    
print('Accuracy of robust model on Purified comb: '+str(correct / total_samples))

Accuracy of robust model on Purified comb: 0.9205298013245033
